Домашнее задание. 

1. Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции.
2. По образу практики, попробуйте создать искусственный датасет с лишними столбцами. 
Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. 
Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. 
Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то "правильного" решения. Цель - достичь базового понимания проблемы. 
Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!


In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

2. По образу практики, попробуйте создать искусственный датасет с лишними столбцами:
а) Создаём искуственый датасет стоимости поездок, со столбцами: [маршрут], [тариф за км. пути],[расстояние поездки],[цена поездки],[наличие льготы пассажира]. 
б) 


In [86]:
dz_sample = 500

route = np.random.choice(79, dz_sample) + 1 # номер маршрута
tarif = np.random.choice(20, dz_sample) + 4 #тфриф за км. поездки
dist = np.random.choice(100, dz_sample) + 1 #расстояние поездки.
discount = np.random.choice(2, dz_sample) #Признак льготы пассажира

price = (tarif-(2*discount))* dist#стоимость поездки- расчитывается как произведение тарифа на растояние 
#и применяется скидка 2 у.е. при наличии признака льготы (1).

data = pd.DataFrame({'route': route, 'tarif': tarif, 'dist': dist,'discount' : discount, 'price': price})
data.head(10)
#print(data.describe())



,route,tarif,dist,discount,price
0,65,14,35,0,490
1,32,14,54,1,648
2,47,12,8,0,96
3,71,23,77,0,1771
4,36,13,56,0,728
5,10,13,90,1,990
6,78,19,73,0,1387
7,63,18,36,1,576
8,41,23,96,1,2016
9,21,14,73,1,876


In [87]:
from sklearn.metrics import mean_absolute_error

X = data[['route','tarif', 'dist', 'discount']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['route','tarif', 'dist', 'discount']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  -0.67469177   52.78139823   12.3516405  -111.23207992]
Bias: -621.0862862774535
Error: 125.84235516452614


Мы исходим из того, что номер маршрута напрямую не может влиять на стоимость поездки, однако при составлении линейной регрессии мы видим, что модель учитывает вес [route], и показывает смещение-Bias и Error.
Ниже-  убрав, из регрессии параметр маршрута видим что картина (Bias и Error) существено не изменилась. 

In [88]:

X = data[['tarif', 'dist', 'discount']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['tarif', 'dist', 'discount']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  52.70576094   12.33236021 -113.0878584 ]
Bias: -645.2995164091882
Error: 125.82039066269915


Следующим шагом прпробуем "очистить" модель расчётов путём создания нового признака: [full_price] формирующего цену поездки без учёта скидки по льготе.

In [89]:
data['full_price'] = data['tarif'] * data['dist']
data.head()

,route,tarif,dist,discount,price,full_price
0,65,14,35,0,490,490
1,32,14,54,1,648,756
2,47,12,8,0,96,96
3,71,23,77,0,1771,1771
4,36,13,56,0,728,728


Далее делаем линейную регрессию признака [price] по Y и нового признака [full_price] по Х 

In [90]:
X = data[['full_price']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['full_price']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.96203047]
Bias: -27.003434096844103
Error: 54.20108440577264


Таким образом, выявляем и оставляем для работы только те признаки которые максимально зависимы друг от друга и убираем не зависимые признаки([route] никак не влияет и не зависит от [full_price]